# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 11 2022 2:38PM,243987,10,Enova-10472,Emerginnova,Released
1,Aug 11 2022 2:31PM,243986,19,ADV80004876,"AdvaGen Pharma, Ltd",Released
2,Aug 11 2022 2:31PM,243986,19,ADV80004877,"AdvaGen Pharma, Ltd",Released
3,Aug 11 2022 1:51PM,243985,19,ACG-2102490923,ACG North America LLC,Released
4,Aug 11 2022 1:44PM,243984,10,0085935687,ISDIN Corporation,Released
5,Aug 11 2022 1:21PM,243981,10,SOIVA0000514,Ivaoes Animal Health,Released
6,Aug 11 2022 1:21PM,243980,10,SOTAH0000751,"Nextsource Biotechnology, LLC",Released
7,Aug 11 2022 12:59PM,243975,10,8389489,Beach Products Inc,Executing
8,Aug 11 2022 12:59PM,243975,10,8389490,Beach Products Inc,Executing
9,Aug 11 2022 12:59PM,243975,10,8389491,Beach Products Inc,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,243981,Released,1
28,243984,Released,1
29,243985,Released,1
30,243986,Released,2
31,243987,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
243981,NaN,NaN,1.0
243984,NaN,NaN,1.0
243985,NaN,NaN,1.0
243986,NaN,NaN,2.0
243987,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
243907,0.0,0.0,1.0
243912,0.0,0.0,3.0
243913,0.0,0.0,5.0
243933,0.0,17.0,27.0
243934,0.0,0.0,32.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
243907,0.0,0,1
243912,0.0,0,3
243913,0.0,0,5
243933,0.0,17,27
243934,0.0,0,32


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,243907,0.0,0,1
1,243912,0.0,0,3
2,243913,0.0,0,5
3,243933,0.0,17,27
4,243934,0.0,0,32


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,243907,0.0,,1
1,243912,0.0,,3
2,243913,0.0,,5
3,243933,0.0,17,27
4,243934,0.0,,32


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 11 2022 2:38PM,243987,10,Emerginnova
1,Aug 11 2022 2:31PM,243986,19,"AdvaGen Pharma, Ltd"
3,Aug 11 2022 1:51PM,243985,19,ACG North America LLC
4,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation
5,Aug 11 2022 1:21PM,243981,10,Ivaoes Animal Health
6,Aug 11 2022 1:21PM,243980,10,"Nextsource Biotechnology, LLC"
7,Aug 11 2022 12:59PM,243975,10,Beach Products Inc
24,Aug 11 2022 12:57PM,243973,10,ISDIN Corporation
35,Aug 11 2022 12:56PM,243974,10,Amatheon Pharmaceuticals
36,Aug 11 2022 12:49PM,243972,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Aug 11 2022 2:38PM,243987,10,Emerginnova,0.0,,1
1,Aug 11 2022 2:31PM,243986,19,"AdvaGen Pharma, Ltd",0.0,,2
2,Aug 11 2022 1:51PM,243985,19,ACG North America LLC,0.0,,1
3,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation,0.0,,1
4,Aug 11 2022 1:21PM,243981,10,Ivaoes Animal Health,0.0,,1
5,Aug 11 2022 1:21PM,243980,10,"Nextsource Biotechnology, LLC",0.0,,1
6,Aug 11 2022 12:59PM,243975,10,Beach Products Inc,0.0,13,4
7,Aug 11 2022 12:57PM,243973,10,ISDIN Corporation,0.0,,11
8,Aug 11 2022 12:56PM,243974,10,Amatheon Pharmaceuticals,0.0,,1
9,Aug 11 2022 12:49PM,243972,10,Emerginnova,0.0,6,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 2:38PM,243987,10,Emerginnova,1,
1,Aug 11 2022 2:31PM,243986,19,"AdvaGen Pharma, Ltd",2,
2,Aug 11 2022 1:51PM,243985,19,ACG North America LLC,1,
3,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation,1,
4,Aug 11 2022 1:21PM,243981,10,Ivaoes Animal Health,1,
5,Aug 11 2022 1:21PM,243980,10,"Nextsource Biotechnology, LLC",1,
6,Aug 11 2022 12:59PM,243975,10,Beach Products Inc,4,13
7,Aug 11 2022 12:57PM,243973,10,ISDIN Corporation,11,
8,Aug 11 2022 12:56PM,243974,10,Amatheon Pharmaceuticals,1,
9,Aug 11 2022 12:49PM,243972,10,Emerginnova,1,6


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 2:38PM,243987,10,Emerginnova,1,
1,Aug 11 2022 2:31PM,243986,19,"AdvaGen Pharma, Ltd",2,
2,Aug 11 2022 1:51PM,243985,19,ACG North America LLC,1,
3,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation,1,
4,Aug 11 2022 1:21PM,243981,10,Ivaoes Animal Health,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 2:38PM,243987,10,Emerginnova,1.0,NaN
1,Aug 11 2022 2:31PM,243986,19,"AdvaGen Pharma, Ltd",2.0,NaN
2,Aug 11 2022 1:51PM,243985,19,ACG North America LLC,1.0,NaN
3,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation,1.0,NaN
4,Aug 11 2022 1:21PM,243981,10,Ivaoes Animal Health,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 2:38PM,243987,10,Emerginnova,1.0,0.0
1,Aug 11 2022 2:31PM,243986,19,"AdvaGen Pharma, Ltd",2.0,0.0
2,Aug 11 2022 1:51PM,243985,19,ACG North America LLC,1.0,0.0
3,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation,1.0,0.0
4,Aug 11 2022 1:21PM,243981,10,Ivaoes Animal Health,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3659486,142.0,36.0
15,1219649,12.0,3.0
17,731739,15.0,0.0
19,731933,3.0,1.0
20,487924,33.0,3.0
21,243907,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3659486,142.0,36.0
1,15,1219649,12.0,3.0
2,17,731739,15.0,0.0
3,19,731933,3.0,1.0
4,20,487924,33.0,3.0
5,21,243907,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,142.0,36.0
1,15,12.0,3.0
2,17,15.0,0.0
3,19,3.0,1.0
4,20,33.0,3.0
5,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,142.0
1,15,Released,12.0
2,17,Released,15.0
3,19,Released,3.0
4,20,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,17,19,20,21
Status,,,,,,
Executing,36.0,3.0,0.0,1.0,3.0,0.0
Released,142.0,12.0,15.0,3.0,33.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,17,19,20,21
0,Executing,36.0,3.0,0.0,1.0,3.0,0.0
1,Released,142.0,12.0,15.0,3.0,33.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,17,19,20,21
0,Executing,36.0,3.0,0.0,1.0,3.0,0.0
1,Released,142.0,12.0,15.0,3.0,33.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()